In [2]:
# import python libraries

import pandas as pd
import nbconvert
import matplotlib.pyplot as plt
import numpy as np
import scipy
import xlrd
import seaborn as sns
%matplotlib inline
from sklearn.feature_extraction.text import CountVectorizer
import re, string, unicodedata
import os
import glob
import gensim
import nltk
#import contractions
import inflection as inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from string import ascii_lowercase
import itertools, snowballstemmer
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import Word2Vec
import spacy
#import basemap as basemap
from matplotlib.colors import rgb2hex
from matplotlib.patches import Polygon
from geopy.geocoders import Nominatim
import math
a ='a'

C:\Users\yasser\Anaconda2\envs\pyt36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
import spacy
# Load the spacy model that you have installed
nlp = spacy.load('en_core_web_sm')
# process a sentence using the model
doc = nlp("This is some text that I am processing with Spacy")
# It's that simple - all of the vectors and words are assigned after this point
# Get the vector for 'text':
doc[3].vector
# Get the mean vector for the entire sentence (useful for sentence classification etc.)
doc.vector
a='a'

In [19]:
# 1.6  user defined function to calculate the percentage of missing values in each column of the dataset

def nan_percentage(df):
    
    ''' return the percentage of NaN values for each column in the dataframe'''
    
    nandf = df.loc[:, df.isnull().any()]
    
    nan_percent = nandf.isnull().sum(axis = 0)/len(df)*100
    
    return nan_percent
# call the function

nan_percentage(gender_raw_df)

gender                0.483791
gender_confidence     0.129676
description          18.673317
tweet_coord          99.206983
tweet_location       37.361596
user_timezone        38.892768
dtype: float64

In [20]:
''' 
tweet_coord contains 99.206983 percentage of NaN values hence will be dropped

retweet_count contains 96.9 percent zero values, retweet is a measure of popularity of the tweet and could vary with gender

tweet_created is the date of data compilation hence a single variable

tweet_id is also a constant variable and will be dropped

profileimage is a link to website for images which could be used for image processing hence dropped for this model

At the end of the data import and descriptive data analysis, these columns are dropped and raw data is saved to project directory
and two coulmns are dropped.
'''
gender_raw_df2 = gender_raw_df.copy()

gender_raw_df2 = gender_raw_df.drop(['tweet_created','tweet_coord', 'tweet_id', 'profileimage'], axis = 1)

gender_raw_df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20050 entries, 0 to 20049
Data columns (total 14 columns):
unit_id              20050 non-null int64
gender               19953 non-null object
gender_confidence    20024 non-null float64
created              20050 non-null object
description          16306 non-null object
fav_number           20050 non-null int64
link_color           20050 non-null object
name                 20050 non-null object
retweet_count        20050 non-null int64
sidebar_color        20050 non-null object
text                 20050 non-null object
tweet_count          20050 non-null int64
tweet_location       12559 non-null object
user_timezone        12252 non-null object
dtypes: float64(1), int64(4), object(9)
memory usage: 2.3+ MB


In [21]:
''' 2.0 Data cleaning and feature extraction from raw data.

The data is cleaned to remove unwanted symbols and numbers and digits in each column to return a uniform data type in each column.
also considering data set the features such as symbols and characters were extracted before they were cleaned from the column
as required in this data, Feature transformation and engineering will be handled after Data exploration.

'''
go = 'go'

In [22]:
# Data  extraction from raw text in the text variables before cleaning the data 

gender_raw_df2['txword_count'] = gender_raw_df2['text'].apply(lambda x: len(str(x).split(" ")))  # word count

gender_raw_df2['txchar_count'] = gender_raw_df2['text'].str.len() ## this also includes spaces   # character count

def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

gender_raw_df2['txavg_word'] = gender_raw_df2['text'].apply(lambda x: avg_word(x))   # average word length

from nltk.corpus import stopwords
stop = stopwords.words('english')

gender_raw_df2['txstopwords'] = gender_raw_df2['text'].apply(lambda x: len([x for x in x.split() if x in stop])) # number of stop words

gender_raw_df2['txhastags'] = gender_raw_df2['text'].apply(lambda x: len([x for x in x.split() if x.startswith('#')])) # number of hashtags

gender_raw_df2['txnumerics'] = gender_raw_df2['text'].apply(lambda x: len([x for x in x.split() if x.isdigit()])) # number of numerics

gender_raw_df2['txupper'] = gender_raw_df2['text'].apply(lambda x: len([x for x in x.split() if x.isupper()])) # number of uppercase

gender_raw_df2[['text','txword_count','txchar_count','txavg_word','txstopwords','txhastags','txnumerics','txupper']].head() # display dataframe

,text,txword_count,txchar_count,txavg_word,txstopwords,txhastags,txnumerics,txupper
0,Robbie E Responds To Critics After Win Against...,14,109,6.857143,0,1,0,1
1,äóìIt felt like they were my friends and I was...,19,139,6.368421,7,2,0,2
2,i absolutely adore when louis starts the songs...,16,80,4.062500,7,0,0,0
3,Hi @JordanSpieth - Looking at the url - do you...,23,138,5.272727,7,0,0,2
4,Watching Neighbours on Sky+ catching up with t...,12,95,7.000000,4,0,0,0


In [23]:
# extract features from raw description data
# Data  extraction from raw description in the text variable column before cleaning the data for vectorisation 

gender_raw_df3 = gender_raw_df2.copy()
'''fill NaN values in the description column with missing then convert to lower case'''

gender_raw_df3.description = gender_raw_df2.description.fillna('missing')  # fill missing values with 'missing'


gender_raw_df3['dsword_count'] = gender_raw_df3['description'].apply(lambda x: len(str(x).split(" ")))  # word count

gender_raw_df3['dschar_count'] = gender_raw_df3['description'].str.len() ## this also includes spaces   # character count

def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

gender_raw_df3['dsavg_word'] = gender_raw_df3['description'].apply(lambda x: avg_word(x))   # average word length

from nltk.corpus import stopwords
stop = stopwords.words('english')

gender_raw_df3['dsstopwords'] = gender_raw_df3['description'].apply(lambda x: len([x for x in x.split() if x in stop])) # number of stop words

gender_raw_df3['dshastags'] = gender_raw_df3['description'].apply(lambda x: len([x for x in x.split() if x.startswith('#')])) # number of hashtags

gender_raw_df3['dsnumerics'] = gender_raw_df3['description'].apply(lambda x: len([x for x in x.split() if x.isdigit()])) # number of numerics

gender_raw_df3['dsupper'] = gender_raw_df3['description'].apply(lambda x: len([x for x in x.split() if x.isupper()])) # number of uppercase

gender_raw_df3[['description','dsword_count','dschar_count','dsavg_word','dsstopwords','dshastags','dsnumerics','dsupper']].head() #display data


,description,dsword_count,dschar_count,dsavg_word,dsstopwords,dshastags,dsnumerics,dsupper
0,i sing my own rhythm.,5,21,3.400000,3,0,0,0
1,I'm the author of novels filled with family dr...,11,62,4.727273,4,0,0,0
2,louis whining and squealing and all,6,35,5.000000,3,0,0,0
3,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",24,146,6.150000,0,0,0,2
4,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,27,160,4.962963,6,0,0,1


In [24]:
# Add extracted features to the dataframe.

gender_extract_df = gender_raw_df3.copy()

extract_list = ['dsword_count','dschar_count','dsavg_word','dsstopwords','dshastags','dsnumerics','dsupper','txword_count','txchar_count'
                ,'txavg_word','txstopwords','txhastags','txnumerics','txupper']

gender_extract_df = gender_extract_df[extract_list]

gender_extract_df.head(2)

,dsword_count,dschar_count,dsavg_word,dsstopwords,dshastags,dsnumerics,dsupper,txword_count,txchar_count,txavg_word,txstopwords,txhastags,txnumerics,txupper
0,5,21,3.400000,3,0,0,0,14,109,6.857143,0,1,0,1
1,11,62,4.727273,4,0,0,0,19,139,6.368421,7,2,0,2


In [61]:
# 2.1 The text column is a text format data and has to be cleaned.
# In this pre-processing step the text is converted into lower case to avoid having multiple copies of the same word.

gender_clean_df = gender_raw_df.copy() # copy gender_raw_df

gender_clean_df['text_clean'] = gender_raw_df['text'].apply(lambda x: " ".join(x.lower() for x in x.split())) # convert to lower case characters

# Remove punctuation, as it doesn’t add any extra information and will reduce the size of the training data

gender_clean_df['text_clean'] = gender_clean_df['text_clean'].str.replace('[^\w\s]','') # remove punctuation

# further cleaning of non string characters

gender_clean_df.text_clean = gender_clean_df.text_clean.str.replace('[#%\'()*+,-./:;<=>?@\[\]^_`{|}~1234567890’”“′‘\\\]', ' ')  # rseplace given character

gender_clean_df.text_clean = gender_clean_df.text_clean.str.replace(r'[^\x00-\x7f]', '')   # regex replace lines with non ascii characters

gender_clean_df.text_clean = gender_clean_df.text_clean.str.replace('!', ' ')  # replace ! character.

# stop words (or commonly occurring words) should be removed from the text data

stop = stopwords.words('english')

# grab stopword list, extend it a bit, and then turn it into a set for later

stop.extend(['may','also','zero','one','two','three','four','five','six','seven','eight','nine','ten','across',
             'among','beside','however','yet','within']+list(ascii_lowercase))

stoplist = set(stop)
stop = set(sorted(stop + list(stoplist)))

gender_clean_df['text_clean'] = gender_clean_df['text_clean'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))



gender_clean_df['text_clean'].head()  # display the data

0    robbie responds critics win eddie edwards worl...
1    felt like friends living story httpstcoarnge y...
2    absolutely adore louis starts songs hits hard ...
3    hi jordanspieth looking url use ifttt dont typ...
4     watching neighbours sky catching neighbs xxx xxx
Name: text_clean, dtype: object

In [62]:
# let’s check the 10 most frequently occurring words in our text data then take call to remove or retain.

text_word_freq = pd.Series(' '.join(gender_clean_df['text_clean']).split()).value_counts()[:10]
text_word_freq

weather     2709
get         2266
channel     1387
httpstco    1378
updates     1357
im          1272
like        1004
love         733
dont         730
new          702
dtype: int64

In [63]:
# let’s remove these words as their presence will not of any use in classification of our text data.

# freq = list(freq.index)

gender_clean_df['text_clean'] = gender_clean_df['text_clean'].apply(lambda x: " ".join(x for x in x.split() if x not in text_word_freq))

gender_clean_df['text_clean'].head()

0    robbie responds critics win eddie edwards worl...
1    felt friends living story httpstcoarnge yhno r...
2    absolutely adore louis starts songs hits hard ...
3    hi jordanspieth looking url use ifttt typicall...
4     watching neighbours sky catching neighbs xxx xxx
Name: text_clean, dtype: object

In [64]:
#  Remove rarely occurring words from the text. Because they’re so rare, the association between them and other 
# words is dominated by noise.

freq_rare = pd.Series(' '.join(gender_clean_df['text_clean']).split()).value_counts()[-10:]

freq_rare

pennington                               1
kyc                                      1
akdhakskansjajhahahshahahahdmbfdjjsjd    1
mallory                                  1
unipay                                   1
cranberry                                1
httpstcontfrw                            1
cyntv                                    1
muddled                                  1
httpstcofawu                             1
dtype: int64

In [65]:
# replace rare words with a more general form and then this will have higher counts

gender_clean_df['text_clean'] = gender_clean_df['text_clean'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_rare))

gender_clean_df['text_clean'].head()

0    robbie responds critics win eddie edwards worl...
1    felt friends living story httpstcoarnge yhno r...
2    absolutely adore louis starts songs hits hard ...
3    hi jordanspieth looking url use ifttt typicall...
4     watching neighbours sky catching neighbs xxx xxx
Name: text_clean, dtype: object

In [66]:
# Stemming refers to the removal of suffices, like “ing”, “ly”, “s”, etc. by a simple rule-based approach

from nltk.stem import PorterStemmer

st = PorterStemmer()

gender_clean_df['text_clean'] = gender_clean_df['text_clean'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

gender_clean_df['text_clean'].head(5)

0    robbi respond critic win eddi edward worldtitl...
1    felt friend live stori httpstcoarng yhno retir...
2      absolut ador loui start song hit hard feel good
3    hi jordanspieth look url use ifttt typic see a...
4             watch neighbour sky catch neighb xxx xxx
Name: text_clean, dtype: object

In [67]:
gender_clean_df.head(5)

,unit_id,gender,gender_confidence,created,description,fav_number,link_color,name,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone,text_clean
0,815719226,male,1.0000,12/05/13 01:48,i sing my own rhythm.,0,08C2C2,sheezy0,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.590000e+17,main; @Kan1shk3,Chennai,robbi respond critic win eddi edward worldtitl...
1,815719227,male,1.0000,10/01/12 13:51,I'm the author of novels filled with family dr...,68,0084B4,DavdBurnett,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,äóìIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.590000e+17,NaN,Eastern Time (US & Canada),felt friend live stori httpstcoarng yhno retir...
2,815719228,male,0.6625,11/28/14 11:30,louis whining and squealing and all,7696,ABB8C2,lwtprettylaugh,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.590000e+17,clcncl,Belgrade,absolut ador loui start song hit hard feel good
3,815719229,male,1.0000,06/11/09 22:39,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",202,0084B4,douggarland,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.590000e+17,"Palo Alto, CA",Pacific Time (US & Canada),hi jordanspieth look url use ifttt typic see a...
4,815719230,female,1.0000,4/16/14 13:23,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,37318,3B94D9,WilfordGemma,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.590000e+17,NaN,NaN,watch neighbour sky catch neighb xxx xxx


In [68]:

# compute word frequency and remove all words that don't occur at least 5 times
# df.Firm_Name.str.split(expand=True).stack().value_counts()
# pd.Series(np.concatenate([x.split() for x in df.Firm_Name])).value_counts()
# pd.Series(' '.join(df.Firm_Name).split()).value_counts()
# pd.Series(' '.join(df.Firm_Name).split()).value_counts()[:3]

In [69]:
word_freq = gender_clean_df.text_clean.str.split(expand=True).stack().value_counts()[:10]
word_freq

go        889
make      822
day       753
time      709
look      608
peopl     565
know      555
see       554
want      542
follow    527
dtype: int64

In [70]:
#filtered = df.groupby('positions').filter(lambda x: len(x) >= 3)
#filtered
#df[df['positions'].isin(counts[counts > 3].index)]
#filtered = df.groupby('positions')['r vals'].filter(lambda x: len(x) >= 3)
#df[df['r vals'].isin(filtered)]
#df.groupby('positions')['r vals'].filter(lambda x: len(x) >= 3)
# pd.Series(' '.join(df['text']).lower().split()).value_counts()[:100]

In [72]:
# clean 'description' column
gender_clean_df2 = gender_clean_df.copy()

gender_clean_df2.clean_description = gender_clean_df2.description.str.replace('[^\w\s]','')  # remove punctuations

gender_clean_df2.clean_description = gender_clean_df2.clean_description.str.replace('[#%\'()*+,-./:;<=>?@\[\]^_`{|}~1234567890’”“′‘\\\]', ' ')  # rseplace given character

gender_clean_df2.clean_description = gender_clean_df2.clean_description.str.replace(r'[^\x00-\x7f]', '')   # regex replace lines with non ascii characters

gender_clean_df2.clean_description = gender_clean_df2.clean_description.str.replace('!', ' ')  # replace ! character.


'''fill NaN values in the description column with missing then convert to lower case'''

gender_clean_df2.clean_description = gender_clean_df2.clean_description.fillna('missing')  # fill missing values with 'missing'


gender_clean_df2.clean_description = gender_clean_df2.clean_description.apply(lambda x: " ".join(x.lower() for x in x.split()))  # lower case text

gender_clean_df2['clean_description'] = gender_clean_df2.clean_description


gender_clean_df2.head(3)       

,unit_id,gender,gender_confidence,created,description,fav_number,link_color,name,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone,text_clean,clean_description
0,815719226,male,1.0000,12/05/13 01:48,i sing my own rhythm.,0,08C2C2,sheezy0,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.590000e+17,main; @Kan1shk3,Chennai,robbi respond critic win eddi edward worldtitl...,i sing my own rhythm
1,815719227,male,1.0000,10/01/12 13:51,I'm the author of novels filled with family dr...,68,0084B4,DavdBurnett,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,äóìIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.590000e+17,NaN,Eastern Time (US & Canada),felt friend live stori httpstcoarng yhno retir...,im the author of novels filled with family dra...
2,815719228,male,0.6625,11/28/14 11:30,louis whining and squealing and all,7696,ABB8C2,lwtprettylaugh,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.590000e+17,clcncl,Belgrade,absolut ador loui start song hit hard feel good,louis whining and squealing and all


In [73]:
# clean the 'names' column

gender_clean_df3 = gender_clean_df2.copy()

gender_clean_df3.name_clean = gender_clean_df3.name.apply(lambda x: x.translate(string.punctuation))
gender_clean_df3.name_clean = gender_clean_df3.name_clean.apply(lambda x: x.translate(string.digits))
gender_clean_df3.name_clean = gender_clean_df3.name_clean.str.replace('[#%\'()*+,-./:;<=>?@\[\]^_`{|}~1234567890’”“′‘\\\]', '') 


gender_clean_df3.name_clean = gender_clean_df3.name_clean.apply(lambda x: x.lower())

gender_clean_df3['name_clean'] = gender_clean_df3.name_clean

gender_clean_df3.head(2)

,unit_id,gender,gender_confidence,created,description,fav_number,link_color,name,profileimage,retweet_count,...,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone,text_clean,clean_description,name_clean
0,815719226,male,1.0,12/05/13 01:48,i sing my own rhythm.,0,08C2C2,sheezy0,https://pbs.twimg.com/profile_images/414342229...,0,...,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.590000e+17,main; @Kan1shk3,Chennai,robbi respond critic win eddi edward worldtitl...,i sing my own rhythm,sheezy
1,815719227,male,1.0,10/01/12 13:51,I'm the author of novels filled with family dr...,68,0084B4,DavdBurnett,https://pbs.twimg.com/profile_images/539604221...,0,...,äóìIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.590000e+17,NaN,Eastern Time (US & Canada),felt friend live stori httpstcoarng yhno retir...,im the author of novels filled with family dra...,davdburnett


In [74]:
# last three letters of name
name_last = gender_clean_df3.name_clean # name_lower to a list name_last
new_last = [x[-3:] for x in name_last]  # last three letters of name

gender_clean_df3['name_last3word'] = new_last   # name_lastword as a column in dataframe
gender_clean_df3['name_last3word'].head(5)      # show first 5 rows of column name_lastword

0    ezy
1    ett
2    ugh
3    and
4    mma
Name: name_last3word, dtype: object

In [75]:
# count the number of letters in each name. female names may be longer or less than male or brand names
# hence could indicate a pattern using the count of letters in the name
name_len = [len(x) for x in name_last]  # length of the name or count of number of letters in name

gender_clean_df3['name_count'] = name_len
gender_clean_df3['name_count'].head(5)

0     6
1    11
2    14
3    11
4    12
Name: name_count, dtype: int64

In [76]:
'''
it does train on the whole text column, not just the individual rows. Each row is considered to be point in 
vector space of words. So it trains on the whole column text (all rows) and then transforms each row into 
word vector and puts that as the new column value.'''
dd = 'dd'

In [77]:
# The variable link_color is in base 16 (Hex) code with 3001 unique values. The values can be converted to primary
# colors code numbers Red, Green, Blue (RGB)
gender_clean_df4 = gender_clean_df3.copy()

gender_clean_df4['link_color'].head(2)

0    08C2C2
1    0084B4
Name: link_color, dtype: object

In [78]:
# clean link_color column
gender_clean_df4.link_color2 = gender_clean_df4.link_color.str.replace('#', '')  # replace # character.
gender_clean_df4.link_color2 = gender_clean_df4.link_color.str.replace('2.', '2')  # replace . character.
gender_clean_df4.link_color2 = gender_clean_df4.link_color.str.replace('-', '')  # replace # character.
gender_clean_df4.link_color2 = gender_clean_df4.link_color.str.replace('[^\w\s]','')  # remove punctuations

In [79]:
# # pad to 6digit before conversion from hex to RGB color number

gender_clean_df4.lcolor2 = gender_clean_df4.link_color2.apply(lambda x: x.rjust(6, '0'))

gender_clean_df4.lcolor2.head(2)

0    08C2C2
1    0084B4
Name: link_color, dtype: object

In [80]:
gender_clean_df4['lcolor2'] = gender_clean_df4.lcolor2

# create function to convert from hex color code to RGB numbers

def hex_to_rgb(value):
    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))

# apply function to convert to RGB color numbers

gender_clean_df4.lcolor2cd = gender_clean_df4.lcolor2.apply(lambda x: hex_to_rgb(x))

# display the RGB color numbers

gender_clean_df4.lcolor2cd[:2]

0    (8, 194, 194)
1    (0, 132, 180)
Name: link_color, dtype: object

In [81]:
# Add the column into the dataframe

gender_clean_df4['color_rgb'] = gender_clean_df4.lcolor2cd 

gender_clean_df4['color_rgb'].head(4) #display the columns

0      (8, 194, 194)
1      (0, 132, 180)
2    (171, 184, 194)
3      (0, 132, 180)
Name: color_rgb, dtype: object

In [82]:
# convert color_rgb column to python list and then dataframe  
dfList = gender_clean_df4['color_rgb'].tolist()   # convert to python list

color_df = pd.DataFrame(dfList, columns=['color_red', 'color_green', 'color_blue','color_ex'])  # convert to dataframe 

color_df = color_df.drop(['color_ex'], axis=1) # drop coloumn color_ex as it is NaN values


In [83]:
# create a new dataframe and add each column of colors

gender_clean_df5 = gender_clean_df4.copy()
gender_clean_df5['color_red'] = color_df.color_red.tolist()
gender_clean_df5['color_green'] = color_df.color_green.tolist()
gender_clean_df5['color_blue'] = color_df.color_blue.tolist()

# rename color

gender_clean_df5 = gender_clean_df5.rename(index=str, columns={"color_red": "linkcolor_red", "color_green": "linkcolor_green", "color_blue": "linkcolor_blue"})

gender_clean_df5.head(2)

,unit_id,gender,gender_confidence,created,description,fav_number,link_color,name,profileimage,retweet_count,...,text_clean,clean_description,name_clean,name_last3word,name_count,lcolor2,color_rgb,linkcolor_red,linkcolor_green,linkcolor_blue
0,815719226,male,1.0,12/05/13 01:48,i sing my own rhythm.,0,08C2C2,sheezy0,https://pbs.twimg.com/profile_images/414342229...,0,...,robbi respond critic win eddi edward worldtitl...,i sing my own rhythm,sheezy,ezy,6,08C2C2,"(8, 194, 194)",8,194,194
1,815719227,male,1.0,10/01/12 13:51,I'm the author of novels filled with family dr...,68,0084B4,DavdBurnett,https://pbs.twimg.com/profile_images/539604221...,0,...,felt friend live stori httpstcoarng yhno retir...,im the author of novels filled with family dra...,davdburnett,ett,11,0084B4,"(0, 132, 180)",0,132,180


In [84]:
# convert sidebar color to RGB numbers

gender_clean_df5.sidebar_color[:5]  # display the first 5 values of the column

0    FFFFFF
1    C0DEED
2    C0DEED
3    C0DEED
4         0
Name: sidebar_color, dtype: object

In [85]:
# clean up punctuations and meta caharcters in sidebar_colors
gender_clean_df6 = gender_clean_df5.copy()
gender_clean_df6.sidebar_color = gender_clean_df5.sidebar_color.str.replace('[^\w\s]','')  # remove punctuations

# # pad to 6digit before conversion from hex to RGB color number

gender_clean_df6.sidebar_color = gender_clean_df6.sidebar_color.apply(lambda x: x.rjust(6, '0'))

# apply function to convert to RGB color numbers

gender_clean_df6.sidebar_colorrgb = gender_clean_df6.sidebar_color.apply(lambda x: hex_to_rgb(x))


In [86]:
# convert color_rgb column to python list and then dataframe  
sidebarlist = gender_clean_df6.sidebar_colorrgb.tolist()   # convert to python list

sidebar_df = pd.DataFrame(sidebarlist, columns=['sidebar_red', 'sidebar_green', 'sidebar_blue'])  # convert to dataframe 


# create a new dataframe and add each column of colors to gender_clean_df6

gender_clean_df6['sidebar_red'] = sidebar_df.sidebar_red.tolist()
gender_clean_df6['sidebar_green'] = sidebar_df.sidebar_green.tolist()
gender_clean_df6['sidebar_blue'] = sidebar_df.sidebar_blue.tolist()

gender_clean_df6.head(2)

,unit_id,gender,gender_confidence,created,description,fav_number,link_color,name,profileimage,retweet_count,...,name_last3word,name_count,lcolor2,color_rgb,linkcolor_red,linkcolor_green,linkcolor_blue,sidebar_red,sidebar_green,sidebar_blue
0,815719226,male,1.0,12/05/13 01:48,i sing my own rhythm.,0,08C2C2,sheezy0,https://pbs.twimg.com/profile_images/414342229...,0,...,ezy,6,08C2C2,"(8, 194, 194)",8,194,194,255,255,255
1,815719227,male,1.0,10/01/12 13:51,I'm the author of novels filled with family dr...,68,0084B4,DavdBurnett,https://pbs.twimg.com/profile_images/539604221...,0,...,ett,11,0084B4,"(0, 132, 180)",0,132,180,192,222,237


In [87]:
# clean tweet_location column

gender_clean_df6.tweet_location[:5]  # display the first five columns of the column

0    main; @Kan1shk3
1                NaN
2             clcncl
3      Palo Alto, CA
4                NaN
Name: tweet_location, dtype: object

In [88]:
# clean meta characters and convert to word vectors. location gives the city names without full adress that can
# be converted to geographic coordinates

gender_clean_df7 = gender_clean_df6.copy()
gender_clean_df7.tweet_location = gender_clean_df6.tweet_location.str.replace('[^\w\s]','')  # remove punctuations

gender_clean_df7.tweet_location = gender_clean_df7.tweet_location.str.replace(r'[^\x00-\x7f]', '')

gender_clean_df7.tweet_location = gender_clean_df7.tweet_location.str.replace('[#%\'()*+,-./:;<=>?@\[\]^_`{|}~1234567890’”“′‘\\\]', ' ')

'''fill NaN values in tweet_location with 'miss' then convert to lower case'''

gender_clean_df7.tweet_location = gender_clean_df7.tweet_location.fillna('miss')  # fill missing values with 'missing'

# convert to lower case

gender_clean_df7.tweet_location = gender_clean_df7.tweet_location.apply(lambda x: x.lower())


gender_clean_df7.tweet_location[:2]

0    main kan shk 
1             miss
Name: tweet_location, dtype: object

In [89]:
from pyspark.sql.session import SparkSession

# instantiate Spark
spark = SparkSession.builder.getOrCreate()


# create pyspark dataframe

gender_loc_df7 = gender_clean_df7.loc[:,['unit_id','tweet_location']] # select unit_id and tweet_location

spark_loc_df7 = spark.createDataFrame(gender_loc_df7)
# display two rows

spark_loc_df7.show(n=2)


+---------+--------------+
|  unit_id|tweet_location|
+---------+--------------+
|815719226| main kan shk |
|815719227|          miss|
+---------+--------------+
only showing top 2 rows



In [90]:
# tokenise the words

tokenizer = Tokenizer(inputCol="tweet_location", outputCol="words")
loc_wordsData = tokenizer.transform(spark_loc_df7)

loc_wordsData.show(n=2)

+---------+--------------+----------------+
|  unit_id|tweet_location|           words|
+---------+--------------+----------------+
|815719226| main kan shk |[main, kan, shk]|
|815719227|          miss|          [miss]|
+---------+--------------+----------------+
only showing top 2 rows



In [91]:
loc_words_df = loc_wordsData.select('words')

loc_words_df.show(n=2)

+----------------+
|           words|
+----------------+
|[main, kan, shk]|
|          [miss]|
+----------------+
only showing top 2 rows



In [92]:
# Learn a mapping from words to Vectors.

loc_word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="words", outputCol="result")

loc_model = loc_word2Vec.fit(loc_words_df)

loc_result = loc_model.transform(loc_words_df)


loc_result.show(n=3)

+----------------+--------------------+
|           words|              result|
+----------------+--------------------+
|[main, kan, shk]|[-0.0560951127360...|
|          [miss]|[0.13988752663135...|
|        [clcncl]|[0.11411163955926...|
+----------------+--------------------+
only showing top 3 rows



In [93]:
for row in loc_result.take(2):
    print(row)

Row(words=['main', 'kan', 'shk'], result=DenseVector([-0.0561, 0.0123, -0.0186]))
Row(words=['miss'], result=DenseVector([0.1399, -0.1447, -0.0589]))


In [94]:
loc_result_df2 = loc_result.toPandas()
# convert result to list then a dataframe

dfresult = loc_result_df2['result'].tolist()   # convert to python list

result_pd_df = pd.DataFrame(dfresult, columns=['namevec1', 'namevec2', 'namevec3'], index = range(20050)) # to datafram

gender_clean_df7.index = range(20050)

gender_clean_df8 = pd.concat([gender_clean_df7, result_pd_df], axis=1)

gender_clean_df8.head(3)

,unit_id,gender,gender_confidence,created,description,fav_number,link_color,name,profileimage,retweet_count,...,color_rgb,linkcolor_red,linkcolor_green,linkcolor_blue,sidebar_red,sidebar_green,sidebar_blue,namevec1,namevec2,namevec3
0,815719226,male,1.0000,12/05/13 01:48,i sing my own rhythm.,0,08C2C2,sheezy0,https://pbs.twimg.com/profile_images/414342229...,0,...,"(8, 194, 194)",8,194,194,255,255,255,-0.056095,0.012309,-0.018646
1,815719227,male,1.0000,10/01/12 13:51,I'm the author of novels filled with family dr...,68,0084B4,DavdBurnett,https://pbs.twimg.com/profile_images/539604221...,0,...,"(0, 132, 180)",0,132,180,192,222,237,0.139888,-0.144741,-0.058950
2,815719228,male,0.6625,11/28/14 11:30,louis whining and squealing and all,7696,ABB8C2,lwtprettylaugh,https://pbs.twimg.com/profile_images/657330418...,1,...,"(171, 184, 194)",171,184,194,192,222,237,0.114112,0.061452,0.059334


In [95]:
''' The user_timezone showa the timezone of user location at the time of twitt. The timezones change
over a period of time. Some users only indicated the city name which is not the correct time zone, hence converting
the city to cuerrent time zone may be different from the timezone at twitt time. The column was vectorised to derive
3 word vectors'''

gender_clean_df8.user_timezone.head(5)

0                       Chennai
1    Eastern Time (US & Canada)
2                      Belgrade
3    Pacific Time (US & Canada)
4                           NaN
Name: user_timezone, dtype: object

In [96]:
# clean meta characters and convert to word vectors. location gives the city names without full adress that can
# be converted to geographic coordinates

gender_clean_df9 = gender_clean_df8.copy()
gender_clean_df9.user_timezone = gender_clean_df8.user_timezone.str.replace('[^\w\s]','')  # remove punctuations

gender_clean_df9.user_timezone = gender_clean_df9.user_timezone.str.replace(r'[^\x00-\x7f]', '')

gender_clean_df9.user_timezone = gender_clean_df9.user_timezone.str.replace('[#%\'()*+,-./:;<=>?@\[\]^_`{|}~1234567890’”“′‘\\\]', ' ')

'''fill NaN values in tweet_location with 'miss' then convert to lower case'''

gender_clean_df9.user_timezone = gender_clean_df9.user_timezone.fillna('miss')  # fill missing values with 'missing'

# convert to lower case

gender_clean_df9.user_timezone = gender_clean_df9.user_timezone.apply(lambda x: x.lower())

gender_clean_df9.user_timezone.head(3)

0                    chennai
1    eastern time us  canada
2                   belgrade
Name: user_timezone, dtype: object

In [97]:
# create pyspark dataframe
spark_timezone_df9 = gender_clean_df9.loc[:,['unit_id','user_timezone']] # select unit_id and user_timezone

spark_timezone_df9 = spark.createDataFrame(spark_timezone_df9)

# display two rows

spark_timezone_df9.show(n=2)

+---------+--------------------+
|  unit_id|       user_timezone|
+---------+--------------------+
|815719226|             chennai|
|815719227|eastern time us  ...|
+---------+--------------------+
only showing top 2 rows



In [98]:
# tokenise the words

tz_tokenizer = Tokenizer(inputCol="user_timezone", outputCol="tz_words")
tz_wordsData = tz_tokenizer.transform(spark_timezone_df9)

tz_wordsData.show(n=2)

+---------+--------------------+--------------------+
|  unit_id|       user_timezone|            tz_words|
+---------+--------------------+--------------------+
|815719226|             chennai|           [chennai]|
|815719227|eastern time us  ...|[eastern, time, u...|
+---------+--------------------+--------------------+
only showing top 2 rows



In [99]:
# Learn a mapping from words to Vectors.

tz_word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="tz_words", outputCol="tz_result")

tz_model = tz_word2Vec.fit(tz_wordsData)

tz_transform = tz_model.transform(tz_wordsData)

tz_transform.show(n=3)

+---------+--------------------+--------------------+--------------------+
|  unit_id|       user_timezone|            tz_words|           tz_result|
+---------+--------------------+--------------------+--------------------+
|815719226|             chennai|           [chennai]|[0.02757690288126...|
|815719227|eastern time us  ...|[eastern, time, u...|[-0.2873494982719...|
|815719228|            belgrade|          [belgrade]|[0.09708758443593...|
+---------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [100]:
# convert result to pandas, then to pandas list then a dataframe

tz_transform_df = tz_transform.toPandas()

tz_result_list = tz_transform_df['tz_result'].tolist()   # convert to python list

tz_result_df = pd.DataFrame(tz_result_list, columns=['tzvec1', 'tzvec2', 'tzvec3'], index = range(20050)) #to datafram

tz_result_df.head(3)

,tzvec1,tzvec2,tzvec3
0,0.027577,0.016826,-0.057374
1,-0.287349,0.302401,1.356255
2,0.097088,-0.061970,-0.118187


In [101]:
# merge to gender_clean_df9 dataframe

gender_clean_df9.index = range(20050) # index the dataframe for merging

gender_clean_df9 = pd.concat([gender_clean_df9, tz_result_df], axis=1)

gender_clean_df9.head(3)

,unit_id,gender,gender_confidence,created,description,fav_number,link_color,name,profileimage,retweet_count,...,linkcolor_blue,sidebar_red,sidebar_green,sidebar_blue,namevec1,namevec2,namevec3,tzvec1,tzvec2,tzvec3
0,815719226,male,1.0000,12/05/13 01:48,i sing my own rhythm.,0,08C2C2,sheezy0,https://pbs.twimg.com/profile_images/414342229...,0,...,194,255,255,255,-0.056095,0.012309,-0.018646,0.027577,0.016826,-0.057374
1,815719227,male,1.0000,10/01/12 13:51,I'm the author of novels filled with family dr...,68,0084B4,DavdBurnett,https://pbs.twimg.com/profile_images/539604221...,0,...,180,192,222,237,0.139888,-0.144741,-0.058950,-0.287349,0.302401,1.356255
2,815719228,male,0.6625,11/28/14 11:30,louis whining and squealing and all,7696,ABB8C2,lwtprettylaugh,https://pbs.twimg.com/profile_images/657330418...,1,...,194,192,222,237,0.114112,0.061452,0.059334,0.097088,-0.061970,-0.118187


In [102]:
'''The column created is the date and time of the twitt. It shows the year, month, day and minute'''

# rename the column created to date_created

gender_clean_df9 = gender_clean_df9.rename(index=str, columns={"created": "date_created"})

# convert to datetime
gender_clean_df9['date_created'] = pd.to_datetime(gender_clean_df9['date_created'])

# split to year, month, day, hour and second

gender_clean_df9['year'] = gender_clean_df9['date_created'].dt.year.astype(str).tolist()
gender_clean_df9['month'] = gender_clean_df9['date_created'].dt.month.astype(str).tolist()
gender_clean_df9['day'] = gender_clean_df9['date_created'].dt.day.astype(str).tolist()
gender_clean_df9['hour'] = gender_clean_df9['date_created'].dt.hour.astype(str).tolist()
gender_clean_df9['second'] = gender_clean_df9['date_created'].dt.minute.astype(str).tolist()

# drop the column date_created
gender_clean_df9 = gender_clean_df9.drop(['date_created'], axis = 1)

# display the datframe
gender_clean_df9.head(5)

,unit_id,gender,gender_confidence,description,fav_number,link_color,name,profileimage,retweet_count,sidebar_color,...,namevec2,namevec3,tzvec1,tzvec2,tzvec3,year,month,day,hour,second
0,815719226,male,1.0000,i sing my own rhythm.,0,08C2C2,sheezy0,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,...,0.012309,-0.018646,0.027577,0.016826,-0.057374,2013,12,5,1,48
1,815719227,male,1.0000,I'm the author of novels filled with family dr...,68,0084B4,DavdBurnett,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,...,-0.144741,-0.058950,-0.287349,0.302401,1.356255,2012,10,1,13,51
2,815719228,male,0.6625,louis whining and squealing and all,7696,ABB8C2,lwtprettylaugh,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,...,0.061452,0.059334,0.097088,-0.061970,-0.118187,2014,11,28,11,30
3,815719229,male,1.0000,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",202,0084B4,douggarland,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,...,-0.096245,-0.274329,-0.276759,0.303237,1.348049,2009,6,11,22,39
4,815719230,female,1.0000,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,37318,3B94D9,WilfordGemma,https://pbs.twimg.com/profile_images/564094871...,0,000000,...,-0.144741,-0.058950,0.165157,-0.145163,-0.059959,2014,4,16,13,23


In [103]:
# select and drop transformed columns to create final clean dataframe
# The text, description and namelast3word columns will be vecrorised and dropped later

col_drop = ['description', 'link_color', 'name', 'sidebar_color', 'text', 'tweet_location', 'user_timezone', 'lcolor2', 'color_rgb', 'name_clean']

# drop columns
gender_clean_df10 = gender_clean_df9.drop(col_drop, axis = 1)

# display dataframe

gender_clean_df10.iloc[0:2,0:11]


,unit_id,gender,gender_confidence,fav_number,profileimage,retweet_count,tweet_coord,tweet_count,tweet_created,tweet_id,text_clean
0,815719226,male,1.0,0,https://pbs.twimg.com/profile_images/414342229...,0,NaN,110964,10/26/15 12:40,6.590000e+17,robbi respond critic win eddi edward worldtitl...
1,815719227,male,1.0,68,https://pbs.twimg.com/profile_images/539604221...,0,NaN,7471,10/26/15 12:40,6.590000e+17,felt friend live stori httpstcoarng yhno retir...


In [104]:
gender_clean_df10.iloc[0:2,13:40]

,name_count,linkcolor_red,linkcolor_green,linkcolor_blue,sidebar_red,sidebar_green,sidebar_blue,namevec1,namevec2,namevec3,tzvec1,tzvec2,tzvec3,year,month,day,hour,second
0,6,8,194,194,255,255,255,-0.056095,0.012309,-0.018646,0.027577,0.016826,-0.057374,2013,12,5,1,48
1,11,0,132,180,192,222,237,0.139888,-0.144741,-0.058950,-0.287349,0.302401,1.356255,2012,10,1,13,51


In [105]:
# display dataframe information

gender_clean_df10.info()


<class 'pandas.core.frame.DataFrame'>
Index: 20050 entries, 0 to 20049
Data columns (total 31 columns):
unit_id              20050 non-null int64
gender               19953 non-null object
gender_confidence    20024 non-null float64
fav_number           20050 non-null int64
profileimage         20050 non-null object
retweet_count        20050 non-null int64
tweet_coord          159 non-null object
tweet_count          20050 non-null int64
tweet_created        20050 non-null object
tweet_id             20050 non-null float64
text_clean           20050 non-null object
clean_description    20050 non-null object
name_last3word       20050 non-null object
name_count           20050 non-null int64
linkcolor_red        20050 non-null int64
linkcolor_green      20050 non-null int64
linkcolor_blue       20050 non-null int64
sidebar_red          20050 non-null int64
sidebar_green        20050 non-null int64
sidebar_blue         20050 non-null int64
namevec1             20050 non-null float64
nam

In [ ]:
# Exploration Data Analysis